In [2]:
import os
import sys
import time
import datetime
import re
import pickle
import getpass
from bs4 import BeautifulSoup
import pandas as pd
import pandas.io.sql as psql
import pymysql
import numpy as np
import pylab
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
#import seaborn.apionly as sns
import nltk
import nltk.data
from nltk.corpus import stopwords

%matplotlib inline
#%pylab inline

In [4]:
p = getpass.getpass()
conn = pymysql.connect(host='history-lab.org', port=3306, user='de_reader', passwd=p, db='declassification_cables')
cur = conn.cursor()

········


In [5]:
cur.execute("show databases;")
for r in cur.fetchall():
   print(r)

('information_schema',)
('authentication',)
('bookwormDB',)
('clinton_test',)
('clinton_test_2',)
('ddrs_equity',)
('declassification',)
('declassification_api',)
('declassification_api_test',)
('declassification_api_update',)
('declassification_cabinet',)
('declassification_cables',)
('declassification_clinton',)
('declassification_clinton_staging',)
('declassification_cpdoc',)
('declassification_ddrs',)
('declassification_foia_dod',)
('declassification_frus',)
('declassification_frus_update',)
('declassification_kissinger',)
('declassification_pdb',)
('declassification_pdb_test',)
('etc',)
('historylab_user_information',)
('mysql',)
('performance_schema',)
('predict_history',)
('predict_history_new',)
('predictify_source',)
('predictify_target',)
('sys',)
('user_information',)
('visualizations',)


In [6]:
cur.execute("use declassification_cables;")
cur.execute("show tables;")
for r in cur.fetchall():
   print(r)

('classification_countries',)
('classification_doc',)
('classifications',)
('concept_doc',)
('concepts',)
('countries',)
('country_doc',)
('doc_counts',)
('docs',)
('from_to_sum',)
('network_docs',)
('network_nodes',)
('office_doc',)
('offices',)
('person_doc',)
('persons',)
('reference_doc',)
('tag_doc',)
('tag_doc_staging',)
('tagname_doc',)
('tagnames',)
('tags',)
('tags_staging',)
('tokens',)
('top_classifications',)
('top_countries',)
('top_network',)
('top_persons',)
('top_topics',)
('topic_doc',)
('topic_token',)
('topics',)
('urgency',)
('urgency_doc',)
